In [155]:
import pandas as pd

In [156]:
import folium

In [157]:
from shapely.geometry import Point, Polygon

In [158]:
#map centered in winnipeg
m = folium.Map(location=[49.8540, -97.1526], zoom_start=12)

In [159]:
def readOPT(year):
    url = './datasets/Orthographic_Photography_Tiles_' + str(year) + '.csv'
    df = pd.read_csv(url)
    return df

In [160]:
point = Point(49.87674, -97.14463)  # (lat, long), User Input
year = 2021                                       #User Input

if (year in [2021, 2018, 2016, 2014, 2012, 2009, 2005]):
    df = readOPT(year)
LocationCln = list(df.Location)
for i in range(len(LocationCln)):
    LocationCln[i] = LocationCln[i].replace("POLYGON", "").strip().replace("((", "").replace("))", "").split(", ")
    for j in range(len(LocationCln[i])):
        LocationCln[i][j] = LocationCln[i][j].split(" ")
        LocationCln[i][j][0] = round(float(LocationCln[i][j][0]), 5)
        LocationCln[i][j][1] = round(float(LocationCln[i][j][1]), 5)
        temporary_v = LocationCln[i][j][1]
        LocationCln[i][j][1] = LocationCln[i][j][0]
        LocationCln[i][j][0] = temporary_v
        LocationCln[i][j] = (LocationCln[i][j][0], LocationCln[i][j][1])

df['LocationCln'] = LocationCln
df.head(2)

,Year,File Type,File Size,Compression,Resolution,Season,Image File,World File,Location,LocationCln
0,2021,ECW,107MB,1:04,7.5cm,Spring,https://wpgopendata.blob.core.windows.net/orth...,https://wpgopendata.blob.core.windows.net/orth...,POLYGON ((-97.3495399502951 49.808536850369215...,"[(49.80854, -97.34954), (49.81528, -97.34931),..."
1,2021,ECW,118MB,1:04,7.5cm,Spring,https://wpgopendata.blob.core.windows.net/orth...,https://wpgopendata.blob.core.windows.net/orth...,"POLYGON ((-97.3493105683572 49.81527980829813,...","[(49.81528, -97.34931), (49.82202, -97.34908),..."


In [161]:
def findPointTile(point, df):
    locList = list(df['LocationCln'])
    imgFileList = list(df['Image File'])
    for i in range(len(locList)):
        pol = Polygon(locList[i])
        if (point.within(pol)):
            lat = [locList[i][0][0], locList[i][1][0], locList[i][2][0]]
            long = [locList[i][0][1], locList[i][1][1], locList[i][2][1]]
            latAvg = (max(lat) + min(lat)) / 2
            longAvg = (max(long) + min(long)) / 2
            polygonCenter = (latAvg, longAvg)
            #print((imgFileList[i], locList[i], polygonCenter))
            return (imgFileList[i], locList[i], polygonCenter)
    return ("not found", "not found", "not found")

In [162]:
urlTag1, tile1Coords, polyg1Center = findPointTile(point, df)
tooltip = 'Click for img url (.ecw)'
popup1 = '<h1>Your central point</h1><h2>Url to ECW img</h2><h4>'+str(year)+'</h4><strong><a target="_blank" href="'+urlTag1+'">'+ urlTag1+'<a/></strong>'
folium.Marker(location=[polyg1Center[0], polyg1Center[1]], popup=popup1, tooltip=tooltip).add_to(m)

In [163]:
def mark(url, pointArr):
    popup = '<h1>Url to ECW img</h1><h4>'+str(year)+'</h4><strong><a target="_blank" href="'+url+'">'+ url+'<a/></strong>'
    folium.Marker([pointArr[0], pointArr[1]], popup=popup, tooltip=tooltip).add_to(m)

In [164]:
def buildOverlay(polygonCords):
    overlay = {
      "type": "FeatureCollection",
      "features": [
        {
          "type": "Feature",
          "properties": {},
          "geometry": {
            "type": "Polygon",
            "coordinates": [
              [
                [],
                [],
                [],
                [],
                []
              ]
            ]
          }
        }
      ]
    }
    for i in range(len(polygonCords)):
        overlay['features'][0]['geometry']['coordinates'][0][i].append(polygonCords[i][1])
        overlay['features'][0]['geometry']['coordinates'][0][i].append(polygonCords[i][0])
    return overlay

In [165]:
folium.GeoJson(buildOverlay(tile1Coords), name='').add_to(m)

In [166]:
def paintAdjacentTiles(pointCords, polygCords): 
    for i in range(3):
        if(i == 0):
            poiC2, polC2 = paintNext(pointCords, polygCords, 'x+')
            paintNext(poiC2, polC2, 'y+')
            paintNext(poiC2, polC2, 'y-')
        elif(i == 1):
            poiC2, polC2 = paintNext(pointCords, polygCords, 'x-')
            paintNext(poiC2, polC2, 'y+')
            paintNext(poiC2, polC2, 'y-')
        elif(i == 2):
            paintNext(pointCords, polygCords, 'y+')
            paintNext(pointCords, polygCords, 'y-')

In [167]:
def paintNext(polCenter, polygCoords, orientation):
    currPoint = Point(polCenter[0], polCenter[1])
    polyg = Polygon(polygCoords)
    if(orientation == 'x+'):
        while(currPoint.within(polyg)):
            currPoint = Point(currPoint.x + 0.0002, currPoint.y)
        urlTag2, tile2Coords, polyg2Center = findPointTile(currPoint, df)
        mark(urlTag2, polyg2Center)
        folium.GeoJson(buildOverlay(tile2Coords), name='').add_to(m)
        return (polyg2Center, tile2Coords)
    elif(orientation == 'x-'):
        while(currPoint.within(polyg)):
            currPoint = Point(currPoint.x - 0.0002, currPoint.y)
        urlTag2, tile2Coords, polyg2Center = findPointTile(currPoint, df)
        mark(urlTag2, polyg2Center)
        folium.GeoJson(buildOverlay(tile2Coords), name='').add_to(m)
        return (polyg2Center, tile2Coords)
    elif(orientation == 'y+'):
        while(currPoint.within(polyg)):
            currPoint = Point(currPoint.x, currPoint.y + 0.0002)
        urlTag2, tile2Coords, polyg2Center = findPointTile(currPoint, df)
        mark(urlTag2, polyg2Center)
        folium.GeoJson(buildOverlay(tile2Coords), name='').add_to(m)
        return (polyg2Center, tile2Coords)
    elif(orientation == 'y-'):
        while(currPoint.within(polyg)):
            currPoint = Point(currPoint.x, currPoint.y - 0.0002)
        urlTag2, tile2Coords, polyg2Center = findPointTile(currPoint, df)
        mark(urlTag2, polyg2Center)
        folium.GeoJson(buildOverlay(tile2Coords), name='').add_to(m)
        return (polyg2Center, tile2Coords)

In [168]:
paintAdjacentTiles(polyg1Center, tile1Coords)

In [169]:
m.save('map.html')